In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from numpy import random
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from numpy import matlib
import qgrid

In [2]:
df_x = pd.read_csv('Features_X')
df_y = pd.read_csv('Y_labels')
df_y.drop(['Unnamed: 0'],axis=1, inplace = True)
df_x.drop(['Unnamed: 0'],axis=1, inplace = True)
X = pd.DataFrame.to_numpy(df_x)
Y = pd.DataFrame.to_numpy(df_y)

In [3]:
def error_logistic(Y_pred, Y_true):
    error = 0
    for ye, y in zip(Y_pred, Y_true):
        if ye != y:
            error += 1
    error = float(error)/float(np.size(Y_pred))
    
    return error

In [4]:
def K_vecinos(vecinos):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    Error = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
        
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]
        
        #Se normalizan los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)
        
        
        modelo = KNeighborsClassifier(n_neighbors=vecinos)
        modelo.fit(Xtrain,Ytrain)
        
        #Validación con muestras de entrenamiento
        Ytrain_pred = modelo.predict(Xtrain)
        
        #Validación con las muestras de test    
        Yest = modelo.predict(Xtest)
        
        
        Error[j] = error_logistic(Y_pred=Yest,Y_true=Ytest)
        
        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
        EficienciaVal[j] = np.mean(Yest == Ytest)
        j += 1
        
    print("Modelo entrenado con "+ str(vecinos)+ " vecinos")
    
    return np.mean(EficienciaVal), np.std(EficienciaVal),np.mean(Error),np.std(Error)    

In [9]:
import qgrid

randn = np.random.randn
df_types = pd.DataFrame({
    'Numero de vecinos' : pd.Series([1])})#, 2, 3, 4, 5, 6, 7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["Error"] = ""
df_types["Std Error"] = ""
df_types.set_index(['Numero de vecinos'], inplace=True)


In [14]:
i = 0
for vecinos in df_types.index:   
    eficiencia_val, ic_val,error, std_error = K_vecinos(vecinos)
    df_types.loc("Eficiencia en validacion",i) = str(eficiencia_val)
    df_types.loc("Intervalo de confianza",i) = str(ic_val)
    df_types.loc("Error",i) = str(error)
    df_types.loc("Std Error",i) = str(std_error)
    i=i+1

SyntaxError: can't assign to function call (<ipython-input-14-a9a6a7123476>, line 4)

In [ ]:
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget
qgrid_widget.get_changed_df()

In [ ]:
df_types